In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import datetime as dt
import pandas as pd
import numpy as np

from apidata import (
    quandl_stock_data,
    optimize_portfolio,
    closing_prices,
)

In [2]:
stock_selection = [
    "AMZN",
    "PFE",
    "AMD",
    "HAL",
    "PG",
    "COF",    
]

stock_selection

['AMZN', 'PFE', 'AMD', 'HAL', 'PG', 'COF']

In [3]:
portfolio = optimize_portfolio(assets=stock_selection)

[0] Retrieving Stock Data: AMZN
[1] Retrieving Stock Data: PFE
[2] Retrieving Stock Data: AMD
[3] Retrieving Stock Data: HAL
[4] Retrieving Stock Data: PG
[5] Retrieving Stock Data: COF

Optimizing Portfolio Weights >> Simulations: x 5000

Optimized Portfolio Weights:
AMD         0.2361
AMZN        0.5599
COF         0.0095
HAL         0.0899
PFE         0.0321
PG          0.0724
Return      0.2256
Sharpe      0.5129
Variance    0.2488
Name: 2167, dtype: float64


In [4]:
def backtest_portfolio_performance(pfolio):
    exclude = ["Return", "Sharpe", "Variance"]
    assets = [(a,wt) for a, wt in pfolio.items() if a not in exclude]
    
    # Initialize Portfolio Back-Test Performance DataFrame
    back_test = closing_prices(assets[0][0]).set_index("Date")
    back_test = np.log(back_test / back_test.shift(1)).iloc[1:]
    back_test = back_test.apply(lambda x: x * assets[0][1])
    
    for allocation in assets[1:]:
        stock = allocation[0]
        weight = allocation[1]
        print(f'\nTicker: {stock} \tPortfolio Weight: {weight}')
        
        closing_data = closing_prices(stock).set_index("Date")
        pct_return = np.log(closing_data / closing_data.shift(1)).iloc[1:]
        pct_return = pct_return.apply(lambda x: x * weight)
        back_test = pd.merge(back_test, pct_return, on="Date", how="inner")
        print(pct_return.head())
        
    back_test["RTNp"] = back_test.sum(axis=1)
    print("\n[Historic] Portfolio Performance:\n", back_test.head())
    
    return back_test

In [5]:
historic_perform = backtest_portfolio_performance(pfolio=portfolio)


Ticker: AMZN 	Portfolio Weight: 0.5599
                AMZN
Date                
2014-01-03 -0.002157
2014-01-06 -0.003983
2014-01-07  0.006224
2014-01-08  0.005445
2014-01-09 -0.001269

Ticker: COF 	Portfolio Weight: 0.0095
                 COF
Date                
2014-01-03  0.000011
2014-01-06 -0.000093
2014-01-07  0.000085
2014-01-08  0.000070
2014-01-09  0.000000

Ticker: HAL 	Portfolio Weight: 0.0899
                 HAL
Date                
2014-01-03  0.000215
2014-01-06  0.000340
2014-01-07 -0.000215
2014-01-08 -0.001262
2014-01-09  0.000200

Ticker: PFE 	Portfolio Weight: 0.0321
                 PFE
Date                
2014-01-03  0.000063
2014-01-06  0.000032
2014-01-07  0.000199
2014-01-08  0.000219
2014-01-09 -0.000021

Ticker: PG 	Portfolio Weight: 0.0724
                  PG
Date                
2014-01-03 -0.000081
2014-01-06  0.000171
2014-01-07  0.000697
2014-01-08 -0.001057
2014-01-09  0.000162

[Historic] Portfolio Performance:
                  AMD      AMZN    

In [ ]:
historic_perform.tail(10)

In [ ]:
historic_perform.describe()

In [ ]:
x_min = dt.date(2014, 1, 1)
x_max = dt.date(2018, 3, 27)

historic_perform["RTNp"].plot(alpha=0.75)
plt.grid(axis="y", linestyle="dashed")
plt.hlines(.02, x_min, x_max, color="red", linestyle="dashed")
plt.hlines(-.02, x_min, x_max, color="red", linestyle="dashed")
plt.show()

In [ ]:
RTNp = historic_perform["RTNp"]
RTNp.head(10)

In [ ]:
type(RTNp)

In [ ]:
RTNp.map(lambda x: x + 1)
RTNp.head()